In [12]:
import json
import os

In [ ]:
base_dir = os.path.dirname(os.path.abspath(__file__))

original_label_root = os.path.join(base_dir, 'Data','label')
cropped_label_root = os.path.join(base_dir,'CropData2','label')
cropped_img_root = os.path.join(base_dir, 'CropData2','img')

sets = ["train", "val"]
emotions =['anger', 'happy', 'sadness', 'panic']
valid_extensions = (".jpg", ".jpeg", ".png")

In [14]:
def get_base_filename(filename):
    return os.path.splitext(filename)[0]

for dataset_type in sets :
    for emotion in emotions :
        print(f'Processing {dataset_type}-{emotion}')
        
        cropped_img_dir = os.path.join(cropped_img_root, dataset_type, emotion)
        original_json_path = os.path.join(original_label_root, dataset_type, f'{dataset_type}_{emotion}.json')
        output_json_path = os.path.join(cropped_label_root, dataset_type, f'{dataset_type}_crop_{emotion}.json')
        
        with open(original_json_path, 'r', encoding='euc-kr') as f :
            original_labels = json.load(f)
            
        original_dict = {get_base_filename(item['filename']): item for item in original_labels}
        
        new_labels = []
        
        for cropped_filename in os.listdir(cropped_img_dir) :
            if not cropped_filename.lower().endswith(valid_extensions) :
                continue
            
            
            base_name_part = cropped_filename.split('_crop')[0]
            cropped_key = get_base_filename(base_name_part)
            
            if cropped_key not in original_dict :
                print(f'Warning : {cropped_key} not found in original Json')
                continue
            
            label_info = original_dict[cropped_key]
            
            
            #새 라벨에 크볿이지 파일명과 기본정보 넣기
            
            new_label_item = {
                'filename' : cropped_filename,
                'gender' : label_info.get('gender'),
                'age' : label_info.get('age'),
                "faceExp_uploader": label_info.get("faceExp_uploader"),
                "bg_uploader": label_info.get("bg_uploader"),   
            }
            
            new_labels.append(new_label_item)
        # 새 JSON 저장
        os.makedirs(os.path.dirname(output_json_path), exist_ok=True)
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(new_labels, f, ensure_ascii=False, indent=2)
            